# Dados categorizados

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
values = pd.Series(['apple', 'orange', 'apple', 'apple'] * 2)
values

In [ ]:
pd.unique(values)

In [ ]:
pd.value_counts(values)

In [ ]:
values = pd.Series([0, 1, 0, 0] * 2)
dim = pd.Series(['apple', 'orange'])
dim.take(values)

In [ ]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)
df = pd.DataFrame({'fruit': fruits,
                  'basket_id': np.arange(N),
                  'count': np.random.randint(3, 15, size=N),
                  'weight': np.random.uniform(0, 4, size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight'])
df

In [ ]:
fruit_cat = df['fruit'].astype('category')

In [ ]:
fruit_cat

In [ ]:
c = fruit_cat.values
type(c)

In [ ]:
c.categories

In [ ]:
c.codes

In [ ]:
df['fruit'] = df['fruit'].astype('category')

In [ ]:
df.fruit

In [ ]:
my_categories = pd.Categorical(['foo','bar','foo','baz','bar'])
my_categories

In [ ]:
my_categories[2]

In [ ]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1, 2, 1, 2, 1, 2, 0, 0, 1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

In [ ]:
ordered_cat = pd.Categorical.from_codes(codes, categories,
                                       ordered=True)
ordered_cat

In [ ]:
my_cats_2.as_ordered()

In [ ]:
df = pd.DataFrame({'hoje': np.random.randn(6),
                 'ontem': np.random.randn(6),
                 'coisas': ['abajur', 'vento', 'sol', 'lápis', 'sol', 'vento']})
df

In [ ]:
cat = pd.Categorical(df['coisas'])

In [ ]:
cat

In [ ]:
df['coisas'] = pd.Categorical(df['coisas'], ordered=True) #Categorizando uma coluna

In [ ]:
df.groupby('coisas').mean()

In [ ]:
np.random.seed(12345)
draws = np.random.randn(1000)
draws[:5]

In [ ]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins

In [ ]:
bins.codes[:10]

In [ ]:
df['coisas'].dtypes

In [ ]:
bins = pd.Series(bins, name='quartis')

In [ ]:
results = (pd.Series(draws).groupby(bins).agg(['count','min','max']).reset_index())

In [ ]:
results

In [ ]:
N = 10000000
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

In [ ]:
categories = labels.astype('category')

In [ ]:
labels.memory_usage()

In [ ]:
categories.memory_usage()

In [ ]:
%time _ = labels.astype('category')

In [ ]:
s = pd.Series(['a','b','c','d'] * 2)
cat_s = s.astype('category')
cat_s

In [ ]:
cat_s.cat.codes

In [ ]:
cat_s.cat.categories

In [ ]:
true_categories = ['a', 'b', 'c', 'd', 'e']

In [ ]:
cat_s2 = cat_s.cat.set_categories(true_categories)
cat_s2

In [ ]:
cat_s.value_counts()

In [ ]:
cat_s2.value_counts()

In [ ]:
cat_s3 = cat_s[cat_s.isin(['a','b'])]
cat_s3

In [ ]:
cat_s3.cat.remove_unused_categories()

In [ ]:
pd.get_dummies(cat_s)

# GroupBy

In [ ]:
df = pd.DataFrame({'key': ['a','b','c'] * 4,
                  'value': np.arange(12.)})
df

In [ ]:
g = df.groupby('key').value
g.mean()

In [ ]:
g.transform('mean')

In [ ]:
g.transform(lambda x: x * 2)

In [ ]:
g.transform(lambda x: 2 * x + 13)

In [ ]:
g.transform(lambda x: x * x + 4 * x - 7)

In [ ]:
g.transform(lambda x: x.rank(ascending=False))

In [ ]:
def normalize(x):
    return(x - x.mean()) / x.std()

In [ ]:
g.transform(normalize)

In [ ]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')

In [ ]:
normalized

In [ ]:
N = 15
times = pd.date_range('2017-05-20 00:00', freq='1min', periods=N)
df = pd.DataFrame({'time': times,
                  'value': np.arange(N)})
df

In [ ]:
df.set_index('time').resample('5min').count()

In [ ]:
df2 = pd.DataFrame({'time': times.repeat(3),
                   'key': np.tile(['a','b','c'], N),
                   'value': np.arange(N * 3.)})
df2[:7]

In [ ]:
time_key = pd.Grouper(freq='5min')

In [ ]:
resampled = (df2.set_index('time').groupby(['key', time_key]).sum())
resampled

In [ ]:
resampled.reset_index()

# Encadeamento de métodos

In [ ]:
df = pd.DataFrame({'key': ['a','b','c'] * 2,
                  'col1': np.random.randn(6),
                  'col2': np.random.randn(6)})
df2 = df[df['col2'] < 0]
df2['col1_demeaned'] = df2['col1'] - df2['col1'].mean()
result = df2.groupby('key').col1_demeaned.std()

In [ ]:
df

In [ ]:
v = ['e'] * 6
df3 = df.copy()
df3['key'] = v
df3

In [ ]:
df3 = df.assign(key=v)
df3

In [ ]:
result = (df2.assign(col1_demeaned = df2.col1 - df2.col1.mean())
         .groupby('key')
         .col1_demeaned.std())

In [ ]:
result

# pipe

In [ ]:
def f(x):
    return x=='a'

def g(x):
    return x < 3

In [ ]:
(df.pipe(f)
   .pipe(g))
